# FIR Resampling
<div align="right"><a href="https://people.epfl.ch/paolo.prandoni">Paolo Prandoni</a>, <a href="https://www.epfl.ch/labs/lcav/">LCAV, EPFL</a></div>

In this notebook we will implement a simple fractional resampler that uses local Lagrange interpolation. The resampler can be used to perform any rational sampling rate change (but beware of aliasing when downsampling!)

In [ ]:
import numpy as np
import IPython
from scipy.io import wavfile

## Local Lagrange interpolation

To perform resampling we will need to compute subsample values and, in order to compute subsample values without resorting to a full interpolation of the input signal, the idea is to fit a local polynomial interpolator arount the reference index $n$ and compute the value _of the polynomial_ at $n+\tau$, where $|\tau| < 1/2$. In the following figure, for instance, a quadratic interpolator is used:

![title](interpolation.png)

To fit the polynomial (whose degree, for reasons of symmetry, should be even) we can use Lagrange polynomials. In this case the approximation is simply: 

$$
  x_L(n; \tau) = \sum_{k=-N}^{N} x[n + k] L_k^{(N)}(\tau)
$$

where $L_k^{(N)}(t)$ is the $k$-th Lagrange polynomial of order $2N$.

## Rational sampling rate change

Given a nominal input sampling rate $F_i$ and an output sampling rate $F_o$, a fractional resampler generates $N_o$ output samples for every $N_i$ input samples where

$$
    \frac{N_o}{N_i} = \frac{F_o}{F_i}
$$
with $N_i, N_o$ coprime for maximum efficiency. So the first thing we need is a simple function that simplifies the ratio of sampling frequencies to its lowest terms. For this we will use Euclid's algorithm:

In [ ]:
def simplify(A, B):
    # Euclid's GCD algorithm
    a = A
    b = B
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return A // a, B // b

We can test the function on the usual CD to DVD sampling rate change and indeed we obtain the familiar 160/147 ratio:

In [ ]:
print(simplify(48000, 44100))

## Fractional resampling

A local fractional resampler works as follows:
 * for each output index $m$, find the closest input index $n$, called the "anchor", so that the distance between input and output instants (in seconds) is less than one half of the input sampling period in magnitude; let $\tau$ be this distance, normalized by the input sampling period; clearly $|\tau| < 1/2$;
 * perform a local Lagrange interpolation of the input around $n$;
 * compute the value of the interpolation at $n +\tau$.
 
For a rational sampling rate change, the values of $\tau$ repeat in a pattern every $N_o$ output point, so that the values of the Lagrange interpolation coefficients can be precomputed; for every block of $N_o$ output points we will use $N_i$ input points. 

The method is best understood graphically: in the figures below, the top line shows the samples $x[n]$ in the original sequence, while the bottom line shows the resampled values $y[n]$. The red dotted lines indicate the intervals for each _input_ sample where $\tau$ less than one half in magnitude; the blue arrows show which input sample is used as an anchor point in the computation of an output sample.

Let's first consider downsampling, with a ratio $N_o/N_i = 4/5$; the operation generates 4 output samples for every 5 input samples; the anchor points are determined like so:

![title](down.png)

As apparent from the figure, since the output rate is less than the input rate, every once in a while an input sample is skipped.

In the following figure, the sgnal is upsampled with a ratio $N_o/N_i = 8/5$ generates 8 output samples for every 5 input samples; since the output rate is larger than the input rate, some of the input samples need to be reused.

![title](up.png)

Mathematically, the anchor points are determined like so:

 * the output sample $y[m]$ occurs at time $t_m = m/F_o$
 * the closest input sample will be $x[n]$ (occurring at time $t_n = n/F_i$) where $n$ is such that $|m/F_o - n/F_i| < 1/2$.
 
The required value for $n$ can be found by setting 
$$
     \left|\mbox{frac}\left(\frac{m}{N_o} - \frac{n}{N_i}\right)\right| < \frac{1}{2}
$$
where `frac()` indicates the fractional part of a number. This yields
$$
    n = \mbox{round}\left(m\frac{N_i}{N_o}\right);
$$
the fractional distance between $y[m]$ and $x[n]$ is given by the time difference normalized by the input's sampling period, that is
$$
    \tau = F_i\left(\frac{m}{F_o} - \frac{n}{F_i}\right) = m\frac{N_i}{N_o} - \mbox{round}\left(m\frac{N_i}{N_o}\right).
$$
Note that $\tau = 0$ every time $m$ is a multiple of $N_o$, which confirms the repetition pattern every $N_o$ output samples.

### Setting up the filters

In practice, we want the resampling to be driven by the input data, so we set up an array of $N_i$ filter _lists;_ for every new input sample we look at the next element in the list (modulo $N_i$) and produce as many output samples as there are filters in the list element.

The following function sets up a set of $N_i$ quadratic interpolation filters for each anchor point:

In [ ]:
def setup_filters(output_rate, input_rate):
    No, Ni = simplify(output_rate, input_rate)
    filterbank = [[] for _ in range(Ni)]
    # while output index spans [0, No-1], the input spans [0, Ni-1]
    for m in range(0, No):
        anchor = int(m * Ni / No + 0.5) 
        tau = (m * Ni / No) - anchor
        filterbank[anchor].append([
            tau * (tau - 1) / 2, 
            (1 - tau) * (1 + tau), 
            tau * (tau + 1) / 2
            ])
    return filterbank

We can test with the examples in the figures above:

In [ ]:
setup_filters(4, 5)

In [ ]:
setup_filters(8, 5)

### The final interpolator
We are now ready to write the full interpolation function:

In [ ]:
def resample(output_rate, input_rate, x):
    No, Ni = simplify(output_rate, input_rate)
    filterbank = setup_filters(No, Ni)
    
    y = np.zeros(No * len(x)) // Ni            
    m = 0
    for n, x_n in enumerate(x[1:-1]):
        for fb in filterbank[n % Ni]:
            y[m] = x[n-1] * fb[0] + x[n] * fb[1] + x[n+1] * fb[2]
            m += 1
    return y

We can now test the resampler on a simple sinusoid; we generate the sinusoid at 44.1 KHz and resample at 48KHz; the pitch should not change:

In [ ]:
x = np.cos(2 * np.pi * 440 / 44100 * np.arange(0, 44100))
IPython.display.Audio(x, rate=44100)

In [ ]:
y = resample(12000, 44100, x)
IPython.display.Audio(y, rate=12000)

We can now test the resampler on an audio file; note how aliasing appears when we downsample too much:

In [ ]:
Fi, x = wavfile.read('oao.wav')
IPython.display.Audio(x, rate=Fi)

In [ ]:
y = resample(48000, Fi, x)
IPython.display.Audio(y, rate=48000)

In [ ]:
y = resample(8000, Fi, x)
IPython.display.Audio(y, rate=8000)